# IMPORTS

In [ ]:
import tensorflow as tf
import os
import cv2
import imghdr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
import random

# LOADING AND VISUALIZATION DATA

In [ ]:
train = pd.read_csv('/handwriting-recognition/written_name_train_v2.csv')
test = pd.read_csv('/handwriting-recognition/written_name_validation_v2.csv')
train_img_dir = '/handwriting-recognition/train_v2/train'
test_img_dir = '/handwriting-recognition/validation_v2/validation'

# DATA CLEANING

In [ ]:
print("Tain set: ",train['IDENTITY'].shape[0])
print("UNREADABLE in train set : ", train['IDENTITY'].isnull().sum())
print("Test set: ", test['IDENTITY'].shape[0])
print("UNREADABLE in validation set : ", test['IDENTITY'].isnull().sum())

In [ ]:
train = train[train['IDENTITY'] != 'UNREADABLE']
train = train.dropna()
test = test[test['IDENTITY'] != 'UNREADABLE']
test = test.dropna()
print("train:",train.shape[0]," test:",test.shape[0])

In [ ]:
train_size = 3000
valid_size = 300

# LOADING DATA

In [ ]:

train_data_img = []
train_data_idt = []
#for i in range(test.shape[0]):   #it's take too longe to do all images
for i in range(300):
    image_filename = train['FILENAME'].iloc[i] 
    image_path = os.path.join(train_img_dir,image_filename)
    image = cv2.imread(image_path)
    image = to_binary(image)
    train_data_img.append(image) 
    train_data_idt.append(train['IDENTITY'].iloc[i])   

In [ ]:

test_data_img = []
test_data_idt = []
#for i in range(test.shape[0]):
for i in range(1000):
    image_filename = test['FILENAME'].iloc[i]
    image_path = os.path.join(test_img_dir,image_filename)
    image = cv2.imread(image_path)
    image = to_binary(image)
    test_data_img.append(image)
    test_data_idt.append(test['IDENTITY'].iloc[i])  
    

In [ ]:
for i in range(3):
    print(train_data_idt[i])
    print(train_data_img[i].shape)
    plt.imshow(train_data_img[i],cmap='gray')
    plt.axis('off')
    plt.show()
for i in range(3):
    print(test_data_idt[i])
    print(test_data_img[i].shape)
    plt.imshow(test_data_img[i],cmap='gray')
    plt.axis('off')
    plt.show()

# coding IDENTITY

In [ ]:
all_data = train_data_idt + test_data_idt
# Concatenate names into a single string
all_characters = ''.join(all_data)

# Create a set to get unique characters
unique_characters_set = set(all_characters)

# Convert the set back to a list if needed
unique_characters_list = list(unique_characters_set)

print(unique_characters_list)
print(len(unique_characters_list))

In [ ]:
lb = LabelBinarizer()
lb.fit(unique_characters_list)
lb.classes_
dec = {}
for i in range(len(unique_characters_list)):
    if unique_characters_list[i] in dec.keys():
        pass
    else: 
        a = lb.transform([unique_characters_list[i]])
        a = list(chain(*a)) #flattened_list from [[....]] to []
        dec[unique_characters_list[i]]= a

In [ ]:
print("train data longest word :")
length_of_longest_word_in_train = len(max(train_data_idt, key=len))
print(length_of_longest_word_in_train)
word_with_longest_characters = max(train_data_idt, key=len)
print(word_with_longest_characters)
print()
print("test data longest word :")
length_of_longest_word_in_test = len(max(test_data_idt, key=len))
print(length_of_longest_word_in_test)
word_with_longest_characters = max(test_data_idt, key=len)
print(word_with_longest_characters)

In [ ]:
# Function to convert a name to a list of vectors using the dictionary
def name_to_vectors(name,max_len,num_char):
    zeros_list = [0] * num_char
    num_vec_to_add = max_len - len(name)
    list_vc_name = [dec[char] for char in name]
    
    for i in range(num_vec_to_add):
        list_vc_name.append(zeros_list)
    return list_vc_name

# Creating a list of vectors for each name
vectors_list_train_idt = [name_to_vectors(name,length_of_longest_word_in_train,len(unique_characters_list)) for name in train_data_idt] # for train data
vectors_list_test_idt = [name_to_vectors(name,length_of_longest_word_in_train,len(unique_characters_list)) for name in test_data_idt] # for test data 

#for name, vectors in zip(names, vectors_list):
    #print(f"{name}: {vectors}")
vectors_list_train_idt = np.array(vectors_list_train_idt) #change from list to numpy array
vectors_list_test_idt = np.array(vectors_list_test_idt)
print(vectors_list_train_idt.shape)
print(vectors_list_test_idt.shape)

In [ ]:
#LB = LabelBinarizer()
#train_data_idt = LB.fit_transform(train_data_idt)
#test_data_idt = LB.fit_transform(test_data_idt)
#print(test_data_idt[0:5])

train_data_img = np.array(train_data_img)
train_data_img = train_data_img.reshape(-1,62,284,1)

test_data_img = np.array(test_data_img)
test_data_img = test_data_img.reshape(-1,62,284,1)

#train_data_idt = np.array(train_data_idt)
#test_data_idt = np.array(test_data_idt)

#print(type(train_data_idt[1]), type(test_data_idt))
print(train_data_img.shape,vectors_list_train_idt.shape)
print(test_data_img.shape,vectors_list_test_idt.shape)